### This notebook is for adding poster URL link to each movie record.

* Import the latest-updated csv file named imdbProcessed_{date} and run the code starting at the desired index

* When you're done, run through the rest of the code and export the new imdbProcessed dataset with today's date.

In [2]:
import pandas as pd
from datetime import datetime

In [11]:
today = datetime.today()
today = pd.to_datetime(today, format='%Y-%m-%d')
today

Timestamp('2024-10-09 18:58:19.664913')

In [18]:
imdbLast = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ignoredFiles/TMDB_all_movies.csv",usecols=["title", "imdb_id", "release_date", "runtime"])

In [19]:
# Step 1: Convert 'release_date' to datetime and extract the year
imdbLast['release_date'] = pd.to_datetime(imdbLast['release_date'], format='%Y-%m-%d', errors='coerce')

# Step 2: Filter rows where both 'imdb_id' and 'release_date' are valid (non-null)
valid_time_rows = imdbLast[pd.notna(imdbLast['imdb_id']) & pd.notna(imdbLast['release_date'])].copy()

# Step 3: Extract the year and assign it to a new column 'release_year'
valid_time_rows['release_year'] = valid_time_rows['release_date'].dt.year

valid_rows = valid_time_rows[valid_time_rows['release_date'] <= datetime.today()]

# Now valid_rows is your "imdbProcessed" dataframe with valid rows and a 'release_year' column
imdbProcessed = valid_rows



In [29]:
imdbProcessed = imdbProcessed.sort_values(by=["release_year"], ascending= False).reset_index(drop=True).drop(columns=["index"])

In [51]:
latest_processed = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed_2024-10-10.csv", low_memory= False).drop(columns="Unnamed: 0")

In [21]:
from imdb import Cinemagoer

ia = Cinemagoer()

In [ ]:
import time

for index, row in latest_processed[9987:].iterrows():
    movieId = row["imdb_id"][2:]
    retries = 3  # Number of retries
    for attempt in range(retries):
        try:
            movie = ia.get_movie(movieId)
            if movie.get("cover url"):
                posterLink = movie.get("cover url")
                latest_processed.loc[index, "poster_link"] = posterLink
            else:
                print(f"Poster not found for movie ID: {movieId}")
            break  # Break the loop if successful

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for movie ID {movieId}: {e}")
            time.sleep(2 ** attempt)  # Exponentially backoff
            if attempt == retries - 1:
                print(f"Max retries reached for movie ID {movieId}. Skipping...")
                continue

In [61]:
latest_processed

,title,release_date,runtime,imdb_id,release_year,poster_link
0,La Suprema,2024-05-02,83.0,tt20158934,2024,https://m.media-amazon.com/images/M/MV5BYzc0N2...
1,Honeymoon,2024-01-12,90.0,tt19721342,2024,https://m.media-amazon.com/images/M/MV5BN2E1MG...
2,Johnny & Me - A Journey through Time with John...,2024-01-25,104.0,tt15807040,2024,https://m.media-amazon.com/images/M/MV5BNjIwYz...
3,Milk & Serial,2024-08-08,62.0,tt22075376,2024,https://m.media-amazon.com/images/M/MV5BMmVhMD...
4,Burnt Country,2024-02-21,17.0,tt31224230,2024,https://m.media-amazon.com/images/M/MV5BMDdjZW...
...,...,...,...,...,...,...
554394,Sallie Gardner at a Gallop,1878-06-19,1.0,tt2221420,1878,NaN
554395,L'Amazone,1878-05-06,1.0,tt32173050,1878,NaN
554396,"Zim, Boum, Boum",1878-05-07,1.0,tt16763774,1878,NaN
554397,La Glissade,1878-05-06,1.0,tt32172647,1878,NaN


In [62]:
today = datetime.today().date().strftime('%Y-%m-%d')
today

'2024-10-10'

In [63]:
latest_processed.to_csv(f"/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed_{today}.csv")